In [70]:
import numpy as np
import pandas as pd

from loguru import logger

In [71]:
def read_csv(path: str) -> pd.DataFrame:
    file = pd.read_csv(path)
    return file

TRAIN_SET = read_csv("./train.csv")
TEST_SET = read_csv("./test.csv")
TRAIN_SET.head()

,id,text,emotions
0,27383,i feel awful about it too because it s my job ...,sadness
1,110083,im alone i feel awful,sadness
2,140764,ive probably mentioned this before but i reall...,joy
3,100071,i was feeling a little low few days back,sadness
4,2837,i beleive that i am much more sensitive to oth...,love


In [72]:
dataset = TRAIN_SET
dataset.emotions.value_counts()

joy         412
sadness     351
anger       161
fear        129
love        106
surprise     41
Name: emotions, dtype: int64

In [73]:
dataset = dataset.sort_values("id")
dataset.head()

,id,text,emotions
37,79,article published,joy
669,90,at end of school function to celebrate leaving...,sadness
308,149,football was a very big deal at my high school,joy
743,181,i almost feel your touch that caress so gentle...,love
166,209,held under water by a large wave and thought i...,fear


In [74]:
dataset_dup = dataset.duplicated(subset=["id"])
index = np.where(dataset_dup==True)
index

(array([ 69, 156, 700, 705, 825]),)

In [75]:
dataset = dataset.drop(index[0])

In [76]:
len(dataset)

1195

In [77]:
# dataset_js = [
#     {
#         "id": 27383,
#         "text": "i feel awful about it too because it s my job to get him in a position to succeed and it just didn t happen here",
#         "emotion": "sadness"
#     },
#     {
#         "id": 110083,
#         "text": "im alone i feel awful	",
#         "emotion": "sadness"
#     }
# ]
# # 
# dataset = dataset_js
# texts = [entry['text'] for entry in dataset_js]
# emotions = [entry['emotion'] for entry in dataset_js]
# texts, emotions

In [78]:
# emotion_set = set(emotions)
# emotion_to_int = {emotion: i for i, emotion in enumerate(emotion_set)}
# int_to_emotion = {i: emotion for i, emotion in enumerate(emotion_set)}
# emotion_to_int, int_to_emotion

In [79]:
import nltk
from nltk.corpus import brown
nltk.download()

KeyboardInterrupt: 

In [ ]:
word2vec = {}
word_set = {}
word_all = []
idx = 0
for dataset in (TRAIN_SET, TEST_SET):
    for text in dataset.text:
        words = text.split()
        for word in words:
            if word not in word2vec:
                word_all.append(word)
                word2vec[word] = idx
                idx += 1
    word_set = set(word_all)
    # logger.debug(idx)
logger.info(len(word2vec))
# logger.info(len(word_set))
    # print(text)
    # break

2023-02-13 14:53:00.609 | INFO     | __main__:<module>:15 - 4895


In [ ]:
text_features_train = []
for text in TRAIN_SET.text:
    features = [0] * len(word2vec)
    words = text.split()
    for word in words:
        features[word2vec[word]] = 1
    text_features_train.append(features)
logger.info(len(text_features_train[0]))


2023-02-13 15:09:05.257 | INFO     | __main__:<module>:8 - 4895
2023-02-13 15:09:05.259 | INFO     | __main__:<module>:9 - 1200


In [ ]:
text_features_test = []
for text in TEST_SET.text:
    features = [0] * len(word2vec)
    words = text.split()
    for word in words:
        features[word2vec[word]] = 1
    text_features_test.append(features)
logger.info(len(text_features_test[0]))
logger.info(len(text_features_test))

2023-02-13 15:09:05.904 | INFO     | __main__:<module>:8 - 4895
2023-02-13 15:09:05.905 | INFO     | __main__:<module>:9 - 800


In [ ]:
emotions = [_ for _ in TRAIN_SET.emotions]
# emotions_test = [_ for _ in TEST_SET.emotions]

# emotion_set = set(emotions)
emotion_set = set(emotions)

emotion_to_int = {emotion: i for i, emotion in enumerate(emotion_set)}
int_to_emotion = {i: emotion for i, emotion in enumerate(emotion_set)}

In [ ]:
emotions = [emotion_to_int[emotion] for emotion in emotions]
train_targets = np.zeros((len(emotions), len(emotion_set)))
for i, emotion in enumerate(emotions):
    train_targets[i, emotion] = 1

In [ ]:
np.array(text_features_train).shape

(1200, 4895)

In [ ]:
weights = np.zeros((len(word2vec), len(emotion_set)))
learning_rate = 0.01
num_epochs = 100
for epoch in range(num_epochs):
    scores = np.dot(np.array(text_features_train), weights)
    predictions = 1 / (1 + np.exp(-scores))
    # logger.debug(np.array(predictions).shape)
    # logger.debug(np.array(train_targets).shape)
    error = train_targets - predictions
    gradient = np.dot(np.array(text_features_train).T, error)
    weights += learning_rate * gradient
    logger.info(f"Epoch = {epoch}")

# Predict emotions for the test data
scores = np.dot(text_features_test, weights)
predictions = 1 / (1 + np.exp(-scores))
predictions = np.round(predictions)
predicted_emotion = int_to_emotion[np.argmax(predictions[0])]

2023-02-13 15:12:26.809 | INFO     | __main__:<module>:12 - Epoch = 0
2023-02-13 15:12:27.490 | INFO     | __main__:<module>:12 - Epoch = 1
2023-02-13 15:12:28.124 | INFO     | __main__:<module>:12 - Epoch = 2
2023-02-13 15:12:28.764 | INFO     | __main__:<module>:12 - Epoch = 3
2023-02-13 15:12:29.396 | INFO     | __main__:<module>:12 - Epoch = 4
2023-02-13 15:12:30.028 | INFO     | __main__:<module>:12 - Epoch = 5
2023-02-13 15:12:30.669 | INFO     | __main__:<module>:12 - Epoch = 6
2023-02-13 15:12:31.312 | INFO     | __main__:<module>:12 - Epoch = 7
2023-02-13 15:12:31.951 | INFO     | __main__:<module>:12 - Epoch = 8
2023-02-13 15:12:32.584 | INFO     | __main__:<module>:12 - Epoch = 9
2023-02-13 15:12:33.215 | INFO     | __main__:<module>:12 - Epoch = 10
2023-02-13 15:12:33.844 | INFO     | __main__:<module>:12 - Epoch = 11
2023-02-13 15:12:34.460 | INFO     | __main__:<module>:12 - Epoch = 12
2023-02-13 15:12:35.123 | INFO     | __main__:<module>:12 - Epoch = 13
2023-02-13 15:12

In [ ]:
pred_emotion = []

scores = np.dot(text_features_test, weights)
predictions = 1 / (1 + np.exp(-scores))
# logger.debug(predictions)
# predictions = np.ones(predictions.shape) * np.argmax(predictions, axis=1)
# len(np.argmax(predictions, axis=1))
pred = np.zeros(predictions.shape)
for idx in range(pred.shape[0]):
    # logger.debug(np.argmax(predictions, axis=1)[idx])
    pred[idx][np.argmax(predictions, axis=1)[idx]] = 1

for idx in range(pred.shape[0]):
    # logger.debug(np.where(pred[idx]==1))
    
    # logger.debug(int_to_emotion[np.where(pred[idx]==1)[0][0]])
    pred_emotion.append(int_to_emotion[np.where(pred[idx]==1)[0][0]])
# predictions = np.argmax(predictions)
# predicted_emotion = int_to_emotion[np.argmax(predictions[0])]


In [ ]:
dataset['emotions'] = pred_emotion
dataset.to_csv("./test_lg.csv")

In [ ]:
dataset

,id,text,emotions
0,139312,ive been feeling more optimistic this week tha...,sadness
1,110362,i suppose i was feeling adventurous and volunt...,joy
2,45691,i feel like when a cycle comes i get all depre...,sadness
3,958,when confronted and in my opinion hassled by t...,anger
4,33882,i seldom feel shaky mid run,fear
...,...,...,...
795,566,that was what i felt when i was finally accept...,joy
796,36236,i take every day as it comes i m just focussin...,sadness
797,76229,i just suddenly feel that everything was fake,sadness
798,131640,im feeling more eager than ever to claw back w...,joy


In [ ]:
def sigmoid(x: float):
    # Logistic regression function (Sigmoid)
    output = 1 / (1 + np.exp(-x))
    return output


def read_csv(path: str) -> pd.DataFrame:
    file = pd.read_csv(path)
    return file

In [ ]:
def data_cleaning(dataset: pd.DataFrame) -> None:
    '''
        Clean the duplicated entries
    '''
    dataset = dataset.sort_values("id")
    dataset_dup = dataset.duplicated(subset=["id"])
    index = np.where(dataset_dup==True)
    # logger.debug(dataset.head())
    dataset = dataset.drop(index[0])
    # logger.debug(dataset.head())
    return dataset

In [ ]:
def data_loading(epoch, num_epochs):
    # Load raw dataset
    TRAIN_PATH = "./train.csv"
    TEST_PATH = "./test.csv"
    train_set = read_csv(TRAIN_PATH)
    test_set = read_csv(TEST_PATH)
    
    # data cleaning
    train_set = data_cleaning(train_set)
    test_set = data_cleaning(test_set)
    
    logger.info(f"After data cleaning, len(train_set) is {len(train_set)}, len(test_set) is {len(test_set)}")

    # Cross validation
    # logger.debug(train_set)
    df_shuffle = train_set.sample(frac=1)
    # logger.debug(df_shuffle)
    logger.info(len(df_shuffle))
    df_size = len(df_shuffle)
    idx_split_left = df_size//num_epochs * (epoch)
    idx_split_right = df_size//num_epochs * (epoch+1)
    train_set = pd.concat([df_shuffle.iloc[:idx_split_left], df_shuffle.iloc[idx_split_right:]])
    val_set = df_shuffle.iloc[idx_split_left:idx_split_right]
    return train_set, val_set, test_set

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    train_set, val_set, test_set, df_shuffle = data_loading(epoch, num_epochs)
    logger.debug(df_shuffle)
    break
    # logger.debug(f"{len(train_set)}, {len(val_set)}, {len(test_set)}")

2023-02-28 22:50:53.624 | INFO     | __main__:data_loading:12 - After data cleaning, len(train_set) is 1195, len(test_set) is 799
2023-02-28 22:50:53.627 | INFO     | __main__:data_loading:18 - 1195
2023-02-28 22:50:53.630 | DEBUG    | __main__:<module>:4 -           id                                               text  emotions
601     3021                        i feel a lil dazed actually  surprise
557   121857  i was rusty but i feel like i flowed ok for mo...       joy
994    97315  i used to think that i was protecting people b...   sadness
1185    7295  i feel weird working out in front of my family...  surprise
463    12560  i can post questions or comments and then sche...   sadness
...      ...                                                ...       ...
12     98659  i mean is on this stupid trip of making the gr...       joy
57     66949  i had my week appointment yesterday and left f...   sadness
1068  109114  i still feel anything but strong but i am also...       joy
79

In [ ]:
train_set

,id,text,emotions
926,136082,im not feeling too clever at the moment,joy
1155,49504,i feel like lately i have been trying to put o...,joy
634,98765,i wanted her to feel the love and freedom with...,sadness
1026,35299,i feel i am being unpleasant boring whining ri...,sadness
750,31997,i got up feeling quite irritable but somehow a...,anger
...,...,...,...
12,98659,i mean is on this stupid trip of making the gr...,joy
57,66949,i had my week appointment yesterday and left f...,sadness
1068,109114,i still feel anything but strong but i am also...,joy
795,77413,i know how you are feeling that you feel hopel...,sadness


In [ ]:
df_shuffle = train_set.sample(frac=1)

In [ ]:
train_set

,id,text,emotions
816,NaN,NaN,NaN
774,NaN,NaN,NaN
419,NaN,NaN,NaN
653,NaN,NaN,NaN
445,NaN,NaN,NaN
...,...,...,...
273,NaN,NaN,NaN
733,NaN,NaN,NaN
967,NaN,NaN,NaN
812,NaN,NaN,NaN


In [ ]:
def feature_rep(train_set, val_set, test_set):
    # Create feature representations for data
    word2vec = {}
    word_set = {}
    word_all = []
    idx = 0
    # logger.debug(train_set)
    for dataset in (train_set, val_set, test_set):
        for text in dataset.text:
            words = text.split()
            for word in words:
                if word not in word2vec:
                    word_all.append(word)
                    word2vec[word] = idx
                    idx += 1
        word_set = set(word_all)
        # logger.debug(idx)
    # logger.info(len(word2vec))
    # logger.info(len(word_set))
        # print(text)
        # break
    return word2vec


def text_features(dataset, embedding):
    text_features = []
    for text in dataset.text:
        features = [0] * len(embedding)
        words = text.split()
        for word in words:
            features[embedding[word]] = 1
        text_features.append(features)
    # logger.info(len(text_features[0]))
    # logger.info(len(text_features))
    return text_features

In [ ]:
TRAIN_PATH = "./train.csv"
TEST_PATH = "./test.csv"
train_set = read_csv(TRAIN_PATH)
test_set = read_csv(TEST_PATH)
val_set = read_csv(TRAIN_PATH)
train_set = data_cleaning(train_set)
val_set = data_cleaning(val_set)
test_set = data_cleaning(test_set)
embedding, _, _, text_features_test,\
        emotion_set, _, _, emotion2int, int2emotion = pre_processing(train_set, val_set, test_set)
# your logistic regression 
weights = np.zeros((len(embedding), len(emotion_set)))
learning_rate = 0.01
num_epochs = 10

errors_train = []
errors_val = []


In [ ]:
def pre_processing(train_set: pd.DataFrame, val_set: pd.DataFrame, test_set: pd.DataFrame):
    
    embedding = feature_rep(train_set, val_set, test_set)
    # Create trianing and testing inputs
    text_features_train = text_features(train_set, embedding)
    text_features_val = text_features(val_set, embedding)
    text_features_test = text_features(test_set, embedding)
    
    # Create labels
    emotions_train = [_ for _ in train_set.emotions]
    emotions_val = [_ for _ in val_set.emotions]

    emotion_set = set(emotions_train)

    emotion2int = {emotion: i for i, emotion in enumerate(emotion_set)}
    int2emotion = {i: emotion for i, emotion in enumerate(emotion_set)}
    
    emotions_train = [emotion2int[emotion] for emotion in emotions_train]
    emotions_val = [emotion2int[emotion] for emotion in emotions_val]
    train_targets = np.zeros((len(emotions_train), len(emotion_set)))
    val_targets = np.zeros((len(emotions_val), len(emotion_set)))
    for i, emotion in enumerate(emotions_train):
        train_targets[i, emotion] = 1
    for i, emotion in enumerate(emotions_val):
        val_targets[i, emotion] = 1

    return embedding, text_features_train, text_features_val, text_features_test,\
        emotion_set, train_targets, val_targets, emotion2int, int2emotion


def post_processing(test_set, pred_emotions):
    output = test_set.copy()
    output['emotions'] = pred_emotions
    output.to_csv("./test_lr.csv")
    return output

In [ ]:
for epoch in range(num_epochs):
    # Cross validation
    train_set, val_set, test_set = data_loading(epoch, num_epochs)
    _, text_features_train, text_features_val, _,\
      _, train_targets, val_targets, _, _ = pre_processing(train_set, val_set, test_set)
    # text_features_train = text_features(train_set, embedding)
    # text_features_val = text_features(val_set, embedding)
    # text_features_test = text_features(test_set, embedding)


    output = np.dot(np.array(text_features_train), weights)
    # Logistic regression function (Sigmoid)
    predictions = sigmoid(output)
    # logger.debug(np.array(predictions).shape)
    # logger.debug(np.array(train_targets).shape)
    error = train_targets - predictions
    errors_train.append(error)
    gradient = np.dot(np.array(text_features_train).T, error)
    weights += learning_rate * gradient
    
        
    # evaluation on val_set
    pred_val = sigmoid(np.dot(np.array(text_features_val), weights))
    error_val = val_targets - pred_val
    # logger.debug(pred_val.shape)
    # logger.debug(val_targets.shape)
    # largest_idx = np.argmax(pred_val)
    # pred = np.eye(pred_val.shape[1])[np.argmax(pred_val, axis=1)]
    # logger.debug(pred)
    # pred[largest_idx] = 1
    errors_val.append(error_val)


    predictions = 1 / (1 + np.exp(-pred_val))
    predictions = np.round(predictions)
    predicted_emotion = int2emotion[np.argmax(predictions[0])]
    val_emotions = int2emotion[np.argmax(val_targets)]
    score = np.sum(predicted_emotion==val_emotions) / len(val_targets)
    logger.info(f"Epoch {epoch}: Accuracy = {score * 100}%")
    # break
logger.debug(errors_train)
logger.debug(errors_val)

2023-02-28 23:37:27.555 | INFO     | __main__:data_loading:12 - After data cleaning, len(train_set) is 1195, len(test_set) is 799
2023-02-28 23:37:27.557 | INFO     | __main__:data_loading:18 - 1195
2023-02-28 23:37:28.488 | INFO     | __main__:<module>:39 - Epoch 0: Accuracy = 0.8403361344537815%
2023-02-28 23:37:28.507 | INFO     | __main__:data_loading:12 - After data cleaning, len(train_set) is 1195, len(test_set) is 799
2023-02-28 23:37:28.509 | INFO     | __main__:data_loading:18 - 1195
2023-02-28 23:37:29.355 | INFO     | __main__:<module>:39 - Epoch 1: Accuracy = 0.0%
2023-02-28 23:37:29.376 | INFO     | __main__:data_loading:12 - After data cleaning, len(train_set) is 1195, len(test_set) is 799
2023-02-28 23:37:29.377 | INFO     | __main__:data_loading:18 - 1195
2023-02-28 23:37:30.216 | INFO     | __main__:<module>:39 - Epoch 2: Accuracy = 0.0%
2023-02-28 23:37:30.241 | INFO     | __main__:data_loading:12 - After data cleaning, len(train_set) is 1195, len(test_set) is 799
202